In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az

np.random.seed(1234)

# --- Preregistration (simulated) ---
prereg = """
Preregistration (simulated):

Hypothesis:
  H1: Treatment increases average score compared to control.
  H0: No difference.

Primary outcome:
  Mean difference in continuous outcome Y.

Analysis plan:
  - Frequentist t-test
  - Linear regression Y ~ group
  - Bayesian posterior and 95% credible interval
  - Replication success defined as:
        sign(original effect) == sign(replication effect)
        AND
        credible intervals overlap substantially.

Exclusion criteria: none planned.
Sample sizes:
  Original: 60 participants
  Replication: 100 participants
"""

print(prereg)


# 1. Generate original experiment

# original study
n_original = 60
true_effect = 0.35

group_original = np.random.binomial(1, 0.5, size=n_original)
y_original = (
    2.0 + 
    0.35 * group_original + 
    np.random.normal(0, 1, size=n_original)
)

df_original = pd.DataFrame({
    "group": group_original,
    "y": y_original
})

df_original.head()

# 2. Analyze original study (frequentist)
model_orig = sm.OLS(df_original["y"], sm.add_constant(df_original["group"])).fit()
print(model_orig.summary())
orig_effect = model_orig.params["group"]

# 3. Bayesian analysis of original
with pm.Model() as model_bayes_orig:
    mu = pm.Normal("mu", 0, 5)
    effect = pm.Normal("effect", 0, 1)
    sigma = pm.HalfNormal("sigma", 1)

    yhat = mu + effect * df_original["group"].values
    y_obs = pm.Normal("y_obs", mu=yhat, sigma=sigma, observed=df_original["y"])

    trace_orig = pm.sample(2000, tune=2000, chains=4, target_accept=0.9)

az.plot_posterior(trace_orig, var_names=["effect"])

# 4. Generate replication study

# Preregistrato: n=100
# Effetto reale identico → 0.35

n_replication = 100

group_rep = np.random.binomial(1, 0.5, size=n_replication)
y_rep = (
    2.0 + 
    0.35 * group_rep + 
    np.random.normal(0, 1, size=n_replication)
)

df_rep = pd.DataFrame({"group": group_rep, "y": y_rep})

# 5. Analyze replication (frequentist)

model_rep = sm.OLS(df_rep["y"], sm.add_constant(df_rep["group"])).fit()
print(model_rep.summary())
rep_effect = model_rep.params["group"]

# 6. Bayesian rep + hierarchical synthesis
with pm.Model() as model_hier:
    mu = pm.Normal("mu", 0, 5)
    true_eff = pm.Normal("true_eff", 0, 1)

    sigma_o = pm.HalfNormal("sigma_o", 1)
    sigma_r = pm.HalfNormal("sigma_r", 1)

    yhat_o = mu + true_eff * df_original["group"].values
    yhat_r = mu + true_eff * df_rep["group"].values

    y_o = pm.Normal("y_o", mu=yhat_o, sigma=sigma_o, observed=df_original["y"])
    y_r = pm.Normal("y_r", mu=yhat_r, sigma=sigma_r, observed=df_rep["y"])

    trace_hier = pm.sample(2000, tune=2000, chains=4, target_accept=0.9)

az.plot_forest(trace_hier, var_names=["true_eff"])

# 7. Replication Success Criteria
crit1 = np.sign(orig_effect) == np.sign(rep_effect)
crit2 = (trace_orig.posterior["effect"].mean() * trace_hier.posterior["true_eff"].mean()) > 0
crit3 = abs(orig_effect - rep_effect) < 0.25  # arbitrary tolerance

print("Criteria met? ", crit1 and crit2 and crit3)

ModuleNotFoundError: No module named 'pymc'